> 방향성

1. 전체 리뷰 데이터에 대해 빈도수 측정으로 50개의 단어 추출
2. 해당 단어들을 차원으로 이 맥주의 해당 단어들이 Aroma에 미치는 영향도를 측정
ex. pour이란 단어는 Aroma의 평가에 50개 중 첫번째로(0.x 만큼의) 많은 영향를 끼침
3. 다른 맥주들도 진행하면 
ex. 이 맥주에 pour이란 단어는 Aroma의 평가에 50개 중 열번째로(0.xx만큼) 의 영향을 끼침
4. 사용자가 Aroma와 단어를 선택하면 -> 해당 단어의 Aroma에 대한 영향도가 제일 높았던 Top3 맥주를 추천
ex. 나는 Aroma가 중요한데 맛에 이 단어가 중요해 ~ 그 단어를 누르면 그 단어 영향도가 제일 높았던 맥주 3개를 보여줌 -> 그러면 Aroma 중 gold한 맥주가 뭔지 알려줄 수 있을듯.


- 이렇게 진행하려면 Aroma 등 4가지 지표에 50개의 단어 셋을 서로 다르게 해야될듯 <br>
각 4가지 지표별로 대표되는 단어셋을 가지게 되면 좀 더 관련있는 맥주추천이 가능할 듯

- 각 맥주별로 진행한 후에 영향도를 저장하고, 한 곳에 모은 다음 그 저장된 단어의 영향도를 뽑아서 쓸 수 있도록 해야됨

In [2]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
beer_data = pd.read_csv('/Users/macbookair/졸업작품/July/beer_tok.csv')

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [18]:
beer_data.head(1)

,Unnamed: 0,beer_name,ID,rating,date,review,Aroma,Appearance,Flavor,Mouthfeel,beer_st,alcohol,tokens,tokens_filtered,beer,head,light,white,sweet,malt,good,like,nice,yellow,golden,body,medium,pours,lager,bitter,pale,carbonation,clear,much,banana,wheat,malty,bitterness,thin,drink,slightly,watery,dark,better,citrus,refreshing,great,really,hint,palate,well,creamy,yeast,smooth,quite,note,brown,nothing,orange,nose,crisp,corn,pretty,grain
0,0,Kloud Original Gravity,"vinivini(9,584)",2.9,"December 29, 2022","Straight from the can. Clean, malty, bready & grassy. Light sweet, light to medium bitter. Mouth feel: light to medium body, avera to lively carbonation.",2,4,3,4,Pale Lager - International / Premium,5.0%,"['straight', 'can', 'clean', 'malty', 'bready', 'grassy', 'light', 'sweet', 'light', 'medium', 'bitter', 'mouth', 'feel', 'light', 'medium', 'body', 'avera', 'lively', 'carbonation']","[straight, clean, malty, bready, grassy, light, sweet, light, medium, bitter, mouth, feel, light, medium, body, avera, lively, carbonation]",0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Top50 단어 (빈도수) 로 뽑기

In [6]:
from collections import Counter
import ast

beer_data['tokens_filtered'] = beer_data['tokens_filtered'].apply(ast.literal_eval)

all_tokens = [token for sublist in beer_data['tokens_filtered'] for token in sublist]

token_counter = Counter(all_tokens)

top_50_tokens = token_counter.most_common(50)
top_50_tokens


[('beer', 38553),
 ('head', 36970),
 ('light', 28326),
 ('white', 22236),
 ('sweet', 18915),
 ('malt', 17920),
 ('good', 16809),
 ('like', 13267),
 ('nice', 12834),
 ('yellow', 12410),
 ('golden', 12375),
 ('body', 12140),
 ('medium', 11819),
 ('pours', 11749),
 ('lager', 10962),
 ('bitter', 10953),
 ('pale', 10929),
 ('carbonation', 10473),
 ('clear', 10192),
 ('much', 8566),
 ('banana', 8226),
 ('wheat', 7812),
 ('malty', 7685),
 ('bitterness', 7611),
 ('thin', 7240),
 ('drink', 7091),
 ('slightly', 7014),
 ('watery', 6982),
 ('dark', 6928),
 ('better', 6854),
 ('citrus', 6326),
 ('refreshing', 6249),
 ('great', 5873),
 ('really', 5745),
 ('hint', 5725),
 ('palate', 5668),
 ('well', 5584),
 ('creamy', 5535),
 ('yeast', 5518),
 ('smooth', 5490),
 ('quite', 5350),
 ('note', 5304),
 ('brown', 5287),
 ('nothing', 5267),
 ('orange', 5111),
 ('nose', 4962),
 ('crisp', 4794),
 ('corn', 4691),
 ('pretty', 4690),
 ('grain', 4517)]

In [7]:
from sklearn.linear_model import LinearRegression
import numpy as np

for token, _ in top_50_tokens:
    beer_data[token] = beer_data['tokens_filtered'].apply(lambda x: 1 if token in x else 0)

grouped = beer_data.groupby('beer_name')

def fit_regression(group):
    X = group[[token for token, _ in top_50_tokens]]
    y = group['Aroma']
    model = LinearRegression().fit(X, y)
    return model.coef_

coefficients = {}
for name, group in grouped:
    if len(coefficients) < 5:
        coefficients[name] = fit_regression(group)
    else:
        break

coefficients


{'Asahi Super Dry': array([ 0.01441617, -0.06961123, -0.06272922,  0.06286055,  0.15267293,
         0.06888149,  0.20095187, -0.09540805,  0.28119017, -0.03505516,
         0.09602095,  0.06057149,  0.02803436,  0.0972873 ,  0.08446388,
         0.03802335, -0.14506216,  0.00598641, -0.00329036, -0.09360202,
         0.22431935,  0.36692409,  0.06280364,  0.098638  , -0.07890004,
        -0.0495688 ,  0.0273088 , -0.11044497,  0.53517318,  0.01034724,
         0.15644071,  0.21742974,  0.01742073, -0.01410008, -0.0199314 ,
        -0.06284288,  0.13041004,  0.10515915,  0.07134395,  0.25188188,
         0.02208959,  0.02028394,  0.08235141, -0.10475282, -0.16502695,
        -0.06304724,  0.07667494, -0.1608615 , -0.05357938, -0.02114055]),
 'Asahi Super Dry Black': array([ 1.34369211e-01,  1.38295301e-01, -2.82469905e-01, -2.72492412e-01,
         1.36436628e-01, -2.43785942e-01, -1.74800532e-01, -2.00569387e-01,
        -4.28131337e-02, -8.07513778e-16, -1.99285033e-14,  1.54654171e-

In [26]:
coefficients_df = pd.DataFrame(coefficients, index=[token for token, _ in top_50_tokens])

coefficients_df.head(3)


,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's
beer,0.014416,0.134369,0.018645,-0.034041,0.024048
head,-0.069611,0.138295,0.184404,0.078926,0.056726
light,-0.062729,-0.282470,0.168924,-0.017116,-0.032518


## 모든 맥주

In [23]:
all_coefficients = grouped.apply(fit_regression).to_dict()

all_coefficients_df = pd.DataFrame(all_coefficients, index=[token for token, _ in top_50_tokens]).transpose()


,beer,head,light,white,sweet,malt,good,like,nice,yellow,golden,body,medium,pours,lager,bitter,pale,carbonation,clear,much,banana,wheat,malty,bitterness,thin,drink,slightly,watery,dark,better,citrus,refreshing,great,really,hint,palate,well,creamy,yeast,smooth,quite,note,brown,nothing,orange,nose,crisp,corn,pretty,grain
Asahi Super Dry,0.014416,-0.069611,-0.062729,0.062861,0.152673,0.068881,0.200952,-0.095408,0.281190,-3.505516e-02,9.602095e-02,0.060571,0.028034,0.097287,0.084464,0.038023,-0.145062,0.005986,-0.003290,-0.093602,2.243193e-01,3.669241e-01,0.062804,0.098638,-0.078900,-0.049569,0.027309,-0.110445,0.535173,0.010347,0.156441,0.217430,0.017421,-0.014100,-0.019931,-0.062843,0.130410,0.105159,0.071344,0.251882,0.022090,0.020284,0.082351,-0.104753,-0.165027,-0.063047,0.076675,-0.160861,-0.053579,-0.021141
Asahi Super Dry Black,0.134369,0.138295,-0.282470,-0.272492,0.136437,-0.243786,-0.174801,-0.200569,-0.042813,-8.075138e-16,-1.992850e-14,0.154654,0.173281,-0.406707,-0.301561,-0.214698,-0.439819,-0.250858,-0.467171,-0.090465,-2.498002e-16,-2.775558e-16,0.418537,-0.287685,0.116845,0.090356,-0.223561,0.182496,0.128997,-0.034709,1.000434,-0.565317,0.436989,-0.015395,0.465770,0.787243,0.314247,-0.123125,0.015537,-0.045674,0.441912,0.038069,-0.004653,0.298612,0.000000,-0.266680,0.096840,-0.276540,-0.033355,0.005260
Bavaria 8.6 (Original),0.018645,0.184404,0.168924,-0.062821,-0.039768,0.052013,-0.149428,-0.022979,0.049277,-3.150962e-01,-7.809902e-03,-0.096831,0.090483,-0.066265,0.103120,0.091904,-0.081222,0.256275,0.034529,0.224700,-2.748420e-01,8.593543e-01,-0.163149,0.068245,0.204858,-0.025804,0.254142,0.235765,0.127388,0.035753,0.360691,-0.335753,0.159893,0.076342,-0.286857,-0.518880,-0.396692,-0.150710,-0.407330,0.110202,0.272387,0.008308,0.115966,-0.137176,0.306859,-0.094791,-0.233706,0.018839,0.091558,-0.021806


In [25]:
all_coefficients_df.head(10)

,beer,head,light,white,sweet,malt,good,like,nice,yellow,golden,body,medium,pours,lager,bitter,pale,carbonation,clear,much,banana,wheat,malty,bitterness,thin,drink,slightly,watery,dark,better,citrus,refreshing,great,really,hint,palate,well,creamy,yeast,smooth,quite,note,brown,nothing,orange,nose,crisp,corn,pretty,grain
Asahi Super Dry,0.014416,-0.069611,-0.062729,0.062861,0.152673,0.068881,0.200952,-0.095408,0.281190,-3.505516e-02,9.602095e-02,0.060571,0.028034,0.097287,0.084464,0.038023,-0.145062,0.005986,-0.003290,-0.093602,2.243193e-01,3.669241e-01,0.062804,0.098638,-0.078900,-0.049569,0.027309,-0.110445,5.351732e-01,0.010347,0.156441,0.217430,0.017421,-0.014100,-0.019931,-0.062843,0.130410,0.105159,0.071344,0.251882,0.022090,0.020284,0.082351,-0.104753,-0.165027,-0.063047,0.076675,-0.160861,-0.053579,-0.021141
Asahi Super Dry Black,0.134369,0.138295,-0.282470,-0.272492,0.136437,-0.243786,-0.174801,-0.200569,-0.042813,-8.075138e-16,-1.992850e-14,0.154654,0.173281,-0.406707,-0.301561,-0.214698,-0.439819,-0.250858,-0.467171,-0.090465,-2.498002e-16,-2.775558e-16,0.418537,-0.287685,0.116845,0.090356,-0.223561,0.182496,1.289970e-01,-0.034709,1.000434,-0.565317,0.436989,-0.015395,0.465770,0.787243,0.314247,-0.123125,0.015537,-0.045674,0.441912,0.038069,-0.004653,0.298612,0.000000,-0.266680,0.096840,-0.276540,-0.033355,0.005260
Bavaria 8.6 (Original),0.018645,0.184404,0.168924,-0.062821,-0.039768,0.052013,-0.149428,-0.022979,0.049277,-3.150962e-01,-7.809902e-03,-0.096831,0.090483,-0.066265,0.103120,0.091904,-0.081222,0.256275,0.034529,0.224700,-2.748420e-01,8.593543e-01,-0.163149,0.068245,0.204858,-0.025804,0.254142,0.235765,1.273884e-01,0.035753,0.360691,-0.335753,0.159893,0.076342,-0.286857,-0.518880,-0.396692,-0.150710,-0.407330,0.110202,0.272387,0.008308,0.115966,-0.137176,0.306859,-0.094791,-0.233706,0.018839,0.091558,-0.021806
Bavaria Pilsener / Premium Beer,-0.034041,0.078926,-0.017116,0.091490,-0.059912,0.094902,-0.011612,-0.117757,-0.124530,-1.550694e-03,-1.456875e-02,-0.029176,-0.101520,-0.051831,0.113708,0.009940,-0.015483,0.024927,-0.056297,-0.076206,5.117176e-01,3.862098e-01,0.036444,-0.056658,0.080176,0.128787,0.005171,-0.177253,-3.071069e-01,-0.011102,0.130554,-0.064024,0.060414,0.088722,-0.002098,-0.029168,0.140361,0.135026,-0.324243,-0.074992,-0.024887,0.260694,-0.042150,-0.068534,-0.057589,-0.033761,-0.050519,0.013167,0.010555,-0.169386
Beck's,0.024048,0.056726,-0.032518,-0.057774,-0.015376,-0.016495,0.014941,0.066096,-0.045507,-2.461248e-02,5.966162e-02,0.094914,0.022020,-0.019899,-0.042179,0.098040,-0.032059,0.041523,-0.063025,-0.059937,-5.803386e-01,-4.096289e-02,0.041208,0.112440,0.026539,0.004238,0.098919,0.015423,2.685007e-01,0.049392,0.001354,0.069703,-0.059924,0.021254,0.090532,-0.061378,-0.024524,-0.011776,-0.027059,0.164371,-0.052484,0.043291,-0.242517,0.057718,0.683755,-0.134381,-0.023449,-0.044505,0.074871,0.013611
Berliner Kindl Jubiläums Pilsener Premium,0.057039,-0.034503,0.197437,-0.048191,-0.181280,0.033713,-0.020187,0.155378,-0.073471,-1.661472e-01,-5.752408e-02,0.083459,-0.005157,0.130898,-0.247878,0.027299,0.097631,-0.003640,0.189118,0.001625,6.661338e-16,-1.133244e+00,-0.174536,0.097485,-0.090099,-0.559010,0.056491,0.199825,1.977585e-16,0.133996,-0.241032,-0.433210,-0.546590,-0.411620,-0.146565,0.180511,0.119874,0.460689,-0.093535,-0.331563,-0.359757,0.340680,2.194239,-0.155591,-0.277759,-0.064842,0.035928,-0.066068,0.200298,-0.256797
Bintang Pilsener,0.129354,-0.009505,0.094570,0.073980,0.063054,0.089845,-0.027406,-0.117953,0.518753,-2.571508e-01,-1.609177e-01,-0.004611,-0.058541,0.033923,0.068044,-0.222479,-0.080027,0.094546,0.016501,-0.013597,1.095964e+00,6.413416e-01,0.160981,-0.047990,-0.015654,-0.241663,0.006577,-0.242341,7.321190e-03,0.181067,0.719612,0.142542,-0.289680,0.103962,-0.219518,0.146317,-0.168258,0.017273,1.306626,-0.011204,0.164270,-0.185299,-0.108014,-0.241678,-0.383253,-0.037641,0.171083,-0.148158,-0.039192,0.006891
Brooklyn Special Effects (Hoppy Amber/Lager),-0.158458,-0

맥주 추천

In [22]:
def recommend_beers_based_on_token(token, df, top_n=3):
    sorted_beers = df.sort_values(by=token, ascending=False)
    return sorted_beers.index[:top_n].tolist()

recommended_beers = recommend_beers_based_on_token("sweet", all_coefficients_df)
recommended_beers

['Kloud Fitz Super Clear',
 'Brooklyn Special Effects (Hoppy Amber/Lager)',
 'Hite D (Dry Finish)']

### 다른 3가지 지표들도 데이터 저장

#### Appearance

In [34]:
from sklearn.linear_model import LinearRegression
import numpy as np

for token, _ in top_50_tokens:
    beer_data[token] = beer_data['tokens_filtered'].apply(lambda x: 1 if token in x else 0)

grouped = beer_data.groupby('beer_name')

def fit_regression_Appearance(group):
    X = group[[token for token, _ in top_50_tokens]]
    y = group['Appearance']
    model = LinearRegression().fit(X, y)
    return model.coef_

coefficients_Appearance = {}
for name, group in grouped:
    if len(coefficients_Appearance) < 5:
        coefficients_Appearance[name] = fit_regression_Appearance(group)
    else:
        break

coefficients_Appearance


{'Asahi Super Dry': array([ 0.04419105, -0.08376592, -0.09440315,  0.13218771,  0.10160891,
        -0.01757188,  0.16654502, -0.08789503,  0.33174222, -0.10870267,
         0.04342143, -0.0353782 ,  0.0704319 ,  0.00886476,  0.01778948,
         0.03617813, -0.18017311,  0.11043784,  0.00930049,  0.03202155,
        -0.2359157 ,  0.26808384, -0.03238322,  0.02145194, -0.14157493,
         0.05191085, -0.03898185, -0.11112295, -0.21358459, -0.08987609,
         0.05368334,  0.26146025,  0.2346324 ,  0.0265028 ,  0.11881264,
        -0.01169352,  0.05191877,  0.48259306,  0.17127299,  0.0548928 ,
         0.06211798,  0.01975391, -0.21743118, -0.0740992 , -0.0079619 ,
        -0.04055188,  0.20464569, -0.16533514, -0.10621265, -0.02213335]),
 'Asahi Super Dry Black': array([ 7.48523695e-03, -1.90528960e-01, -3.47715953e-01, -6.06823876e-02,
         9.80247556e-03, -7.00474624e-02,  8.10008069e-03, -1.04789811e-01,
         3.06097165e-01, -7.77156117e-16, -1.00978254e-14, -1.26405074e-

In [35]:
all_coefficients_Appearance = grouped.apply(fit_regression_Appearance).to_dict()

all_coefficients_Appearance = pd.DataFrame(all_coefficients_Appearance, index=[token for token, _ in top_50_tokens]).transpose()


In [36]:
all_coefficients_Appearance.head(3)

,beer,head,light,white,sweet,malt,good,like,nice,yellow,golden,body,medium,pours,lager,bitter,pale,carbonation,clear,much,banana,wheat,malty,bitterness,thin,drink,slightly,watery,dark,better,citrus,refreshing,great,really,hint,palate,well,creamy,yeast,smooth,quite,note,brown,nothing,orange,nose,crisp,corn,pretty,grain
Asahi Super Dry,0.044191,-0.083766,-0.094403,0.132188,0.101609,-0.017572,0.166545,-0.087895,0.331742,-1.087027e-01,4.342143e-02,-0.035378,0.070432,0.008865,0.017789,0.036178,-0.180173,0.110438,0.009300,0.032022,-2.359157e-01,2.680838e-01,-0.032383,0.021452,-0.141575,0.051911,-0.038982,-0.111123,-0.213585,-0.089876,0.053683,0.261460,0.234632,0.026503,0.118813,-0.011694,0.051919,0.482593,0.171273,0.054893,0.062118,0.019754,-0.217431,-0.074099,-7.961902e-03,-0.040552,0.204646,-0.165335,-0.106213,-0.022133
Asahi Super Dry Black,0.007485,-0.190529,-0.347716,-0.060682,0.009802,-0.070047,0.008100,-0.104790,0.306097,-7.771561e-16,-1.009783e-14,-0.012641,0.195583,-0.175975,0.326114,-0.311558,-0.394686,0.412222,-0.242588,0.129401,3.053113e-16,-2.636780e-16,0.368913,-0.175078,0.111824,0.291727,-0.491296,0.446187,-0.026658,0.054642,0.130375,0.567181,-0.466008,-0.110017,0.262198,-0.050695,0.208924,0.147614,0.095934,0.070267,0.130308,-0.191681,0.094392,-0.581285,6.938894e-18,-0.065871,0.694897,0.296145,-0.008196,0.164478
Bavaria 8.6 (Original),-0.095820,0.002071,0.070310,0.034980,-0.037415,-0.054259,-0.022785,0.150171,0.360385,-2.463327e-01,-3.644545e-02,0.120360,0.276505,0.056515,-0.148478,-0.074042,0.063145,-0.375624,0.210713,-0.028929,3.711758e-01,3.898632e-01,0.066121,0.045490,0.023105,-0.363810,0.475040,-0.470784,0.510303,0.317362,0.192323,0.624000,0.102819,-0.008590,0.011242,-0.292824,-0.240745,0.357583,-0.097793,0.414362,0.151737,0.110466,-0.749484,0.049891,5.291822e-02,0.225898,1.850873,0.001576,-0.468437,0.102432


#### Flavor

In [37]:
from sklearn.linear_model import LinearRegression
import numpy as np

for token, _ in top_50_tokens:
    beer_data[token] = beer_data['tokens_filtered'].apply(lambda x: 1 if token in x else 0)

grouped = beer_data.groupby('beer_name')

def fit_regression_Flavor(group):
    X = group[[token for token, _ in top_50_tokens]]
    y = group['Flavor']
    model = LinearRegression().fit(X, y)
    return model.coef_

coefficients_Flavor = {}
for name, group in grouped:
    if len(coefficients_Flavor) < 5:
        coefficients_Flavor[name] = fit_regression_Flavor(group)
    else:
        break

coefficients_Flavor


{'Asahi Super Dry': array([ 0.03280282, -0.03943809, -0.0022007 , -0.04251325,  0.1399234 ,
         0.01372815,  0.27097147, -0.13745961,  0.40052865, -0.09668425,
         0.03088904,  0.03038022, -0.00207829,  0.05354758,  0.08462954,
         0.10189443, -0.16849682,  0.00233963,  0.0455782 , -0.05802513,
         0.13639216,  0.28623528, -0.00730497,  0.07913319, -0.13175506,
         0.02911371, -0.06775298, -0.15446439,  0.13466501, -0.00603616,
         0.16187366,  0.26500106,  0.17780656, -0.02576151, -0.03537461,
        -0.0434556 ,  0.21847189,  0.26638449, -0.06120664,  0.22927594,
         0.01100677,  0.11247426, -0.42772098, -0.11975479, -0.16920555,
        -0.04752491,  0.27403569, -0.21878228,  0.0288567 , -0.06925926]),
 'Asahi Super Dry Black': array([-3.19577290e-02,  1.60378924e-01, -9.41286856e-02,  7.30635019e-03,
         3.42404953e-02, -2.39853206e-01, -2.13772312e-01, -2.58292234e-01,
        -9.75331916e-02, -8.46545056e-16, -2.21721513e-14, -2.63278302e-

In [38]:
all_coefficients_Flavor = grouped.apply(fit_regression_Flavor).to_dict()

all_coefficients_Flavor = pd.DataFrame(all_coefficients_Flavor, index=[token for token, _ in top_50_tokens]).transpose()

In [40]:
all_coefficients_Flavor.head(3)

,beer,head,light,white,sweet,malt,good,like,nice,yellow,golden,body,medium,pours,lager,bitter,pale,carbonation,clear,much,banana,wheat,malty,bitterness,thin,drink,slightly,watery,dark,better,citrus,refreshing,great,really,hint,palate,well,creamy,yeast,smooth,quite,note,brown,nothing,orange,nose,crisp,corn,pretty,grain
Asahi Super Dry,0.032803,-0.039438,-0.002201,-0.042513,0.139923,0.013728,0.270971,-0.137460,0.400529,-9.668425e-02,3.088904e-02,0.030380,-0.002078,0.053548,0.084630,0.101894,-0.168497,0.002340,0.045578,-0.058025,1.363922e-01,2.862353e-01,-0.007305,0.079133,-0.131755,0.029114,-0.067753,-0.154464,0.134665,-0.006036,0.161874,0.265001,0.177807,-0.025762,-0.035375,-0.043456,0.218472,0.266384,-0.061207,0.229276,0.011007,0.112474,-0.427721,-0.119755,-1.692055e-01,-0.047525,0.274036,-0.218782,0.028857,-0.069259
Asahi Super Dry Black,-0.031958,0.160379,-0.094129,0.007306,0.034240,-0.239853,-0.213772,-0.258292,-0.097533,-8.465451e-16,-2.217215e-14,-0.263278,0.266854,-0.301993,0.132828,-0.576336,-0.138503,0.162476,-0.143956,0.362276,-1.942890e-16,-3.053113e-16,0.047225,-0.184788,-0.036588,0.153750,0.090602,0.432269,-0.139821,-0.110640,0.621896,-0.315541,1.090677,0.446787,0.287110,0.487109,0.107001,-0.344386,0.382541,0.238485,-0.182998,-0.079399,0.158261,0.286139,-1.387779e-17,0.037193,-0.038728,-0.819517,0.190860,0.180809
Bavaria 8.6 (Original),0.062631,-0.076968,0.123677,0.050050,0.064797,0.025839,-0.257222,-0.034921,0.043363,-1.600360e-01,1.201518e-01,-0.098090,0.090040,0.017763,0.062392,-0.107995,-0.048005,0.189874,0.031588,0.148646,-1.922813e-01,1.044965e+00,-0.188428,-0.165624,0.006414,-0.052937,0.273549,0.354136,-0.165164,-0.076200,0.307454,-0.439270,0.179593,-0.364385,-0.319710,-0.358196,-0.439785,-0.073743,-0.235033,0.271880,0.192100,-0.150839,-0.268522,-0.102367,8.624987e-01,-0.065448,0.289544,0.092518,0.085643,0.018348


#### Mouthfeel

In [41]:
from sklearn.linear_model import LinearRegression
import numpy as np

for token, _ in top_50_tokens:
    beer_data[token] = beer_data['tokens_filtered'].apply(lambda x: 1 if token in x else 0)

grouped = beer_data.groupby('beer_name')

def fit_regression_Mouthfeel(group):
    X = group[[token for token, _ in top_50_tokens]]
    y = group['Mouthfeel']
    model = LinearRegression().fit(X, y)
    return model.coef_

coefficients_Mouthfeel = {}
for name, group in grouped:
    if len(coefficients_Mouthfeel) < 5:
        coefficients_Mouthfeel[name] = fit_regression_Mouthfeel(group)
    else:
        break

coefficients_Mouthfeel


{'Asahi Super Dry': array([ 0.0213702 , -0.07333048,  0.00672697,  0.03074285,  0.1233488 ,
        -0.02903696,  0.17608277, -0.08282703,  0.2796409 , -0.04458875,
        -0.01073839, -0.04486911,  0.03502545,  0.06961077,  0.06153506,
         0.07385052, -0.16092589,  0.11221035,  0.04170016, -0.0011733 ,
         0.17088132,  0.21767097, -0.09303799,  0.08584704, -0.12835252,
         0.03992925, -0.04982298, -0.1796692 ,  0.48156913,  0.08009908,
         0.06582902,  0.35056144,  0.2715565 , -0.09590712, -0.02483226,
        -0.02566485,  0.20666409,  0.36315199,  0.26396061,  0.22932544,
         0.05620382,  0.04781129, -0.28920544, -0.01707255,  0.29109952,
        -0.12861427,  0.23463613, -0.14702238,  0.05794427, -0.09295645]),
 'Asahi Super Dry Black': array([-1.05835350e-01, -2.54883625e-01, -2.20718726e-01, -4.95563048e-03,
         2.51171660e-01, -2.78174018e-02,  7.01257491e-02, -7.89827776e-02,
        -5.41581069e-01,  3.33066907e-16,  4.71844785e-16,  3.51010015e-

In [42]:
all_coefficients_Mouthfeel = grouped.apply(fit_regression_Mouthfeel).to_dict()

all_coefficients_Mouthfeel = pd.DataFrame(all_coefficients_Mouthfeel, index=[token for token, _ in top_50_tokens]).transpose()


In [66]:
all_coefficients_Mouthfeel.head(3)

,beer,head,light,white,sweet,malt,good,like,nice,yellow,golden,body,medium,pours,lager,bitter,pale,carbonation,clear,much,banana,wheat,malty,bitterness,thin,drink,slightly,watery,dark,better,citrus,refreshing,great,really,hint,palate,well,creamy,yeast,smooth,quite,note,brown,nothing,orange,nose,crisp,corn,pretty,grain
Asahi Super Dry,0.021370,-0.073330,0.006727,0.030743,0.123349,-0.029037,0.176083,-0.082827,0.279641,-4.458875e-02,-1.073839e-02,-0.044869,0.035025,0.069611,0.061535,0.073851,-0.160926,0.112210,0.041700,-0.001173,1.708813e-01,2.176710e-01,-0.093038,0.085847,-0.128353,0.039929,-0.049823,-0.179669,0.481569,0.080099,0.065829,0.350561,0.271556,-0.095907,-0.024832,-0.025665,0.206664,0.363152,0.263961,0.229325,0.056204,0.047811,-0.289205,-0.017073,0.291100,-0.128614,0.234636,-0.147022,0.057944,-0.092956
Asahi Super Dry Black,-0.105835,-0.254884,-0.220719,-0.004956,0.251172,-0.027817,0.070126,-0.078983,-0.541581,3.330669e-16,4.718448e-16,0.351010,0.349132,0.151261,-0.034188,0.277755,-0.221343,-0.166958,0.085191,-0.386108,-3.608225e-16,2.775558e-16,-0.315209,-0.335276,-0.227853,0.433524,-0.094264,-0.719333,-0.078524,0.001953,-0.417842,0.155022,0.427116,0.068740,0.243619,0.089992,0.420711,0.428052,0.649686,0.973241,-0.015260,-0.448164,-0.102077,-0.083353,0.000000,-0.177381,-0.288470,0.229105,-0.282093,0.131709
Bavaria 8.6 (Original),0.138273,-0.060626,0.074718,0.267382,-0.035574,0.123918,-0.070326,0.083692,0.361771,1.768287e-01,-7.561514e-02,-0.074653,0.396429,-0.106356,0.213054,-0.144676,-0.232060,-0.117548,-0.020689,-0.169908,5.658719e-01,7.862915e-01,-0.019237,0.165822,-0.322636,-0.316672,0.255658,-0.300948,-0.080695,-0.067220,-0.053773,0.192416,0.407399,-0.133942,0.149410,-0.221002,-0.190761,0.063007,-0.038247,0.134185,0.033329,0.170692,0.075528,0.056923,0.148564,-0.235537,0.338992,-0.311940,0.124528,0.011644


In [67]:
def select_coefficient_df_simplified():
    dfs = {
        'Aroma': all_coefficients_df,
        'Appearance': all_coefficients_Appearance,
        'Flavor': all_coefficients_Flavor, 
        'Mouthfeel': all_coefficients_Mouthfeel 
    }
    
    df_name = input("Aroma, Appearance, Flavor, Mouthfeel 중 원하는 섹션을 입력해주세요 : ")
    selected_df = dfs.get(df_name, None)
    
    if selected_df is None:
        print("Aroma, Appearance, Flavor, Mouthfeel 중 한가지를 입력해주세요! ")
        return
    
    tokens = [token for token, _ in top_50_tokens]
    print(f"연관 단어 리스트 : {tokens}")
    
    token = input("찾고자 하는 맥주의 연관 단어를 입력해주세요 : ")
    
    if token not in tokens:
        print("해당 단어가 존재하지 않습니다. 리스트에 있는 단어를 입력해주세요! ")
        return
    
    recommended_beers = recommend_beers_based_on_token(token, selected_df)
    print(f" '{df_name}' 섹션 중 단어 '{token}' 와 관련된 추천 맥주입니다. : {recommended_beers}")

select_coefficient_df_simplified()


연관 단어 리스트 : ['beer', 'head', 'light', 'white', 'sweet', 'malt', 'good', 'like', 'nice', 'yellow', 'golden', 'body', 'medium', 'pours', 'lager', 'bitter', 'pale', 'carbonation', 'clear', 'much', 'banana', 'wheat', 'malty', 'bitterness', 'thin', 'drink', 'slightly', 'watery', 'dark', 'better', 'citrus', 'refreshing', 'great', 'really', 'hint', 'palate', 'well', 'creamy', 'yeast', 'smooth', 'quite', 'note', 'brown', 'nothing', 'orange', 'nose', 'crisp', 'corn', 'pretty', 'grain']
 'Flavor' 섹션 중 단어 'head' 와 관련된 추천 맥주입니다. : ['OB Premier Pilsner', 'Hite D (Dry Finish)', 'Hite Extra Cold']
